![](./Untitled.png)

In [1]:
import numpy as np
import scipy.optimize as spo

v0 = float(input('Введите начальную скорость'))
theta = np.radians(int(input('Введите угол броска - от горизонта')))
phi = lambda x: 0
g = 10

res_func = lambda t: phi(v0 * np.cos(theta) * t) - v0*np.sin(theta)*t + g*t**2/2
starting_estimate = 1
ans = spo.fsolve(res_func, starting_estimate)
while ans == 0:
    starting_estimate *= 2
    ans = spo.fsolve(res_func, starting_estimate)
print('Результат:', ans)

Результат: [1.55563492]


# 1 Аналитический метод

## Условие

Дана система уравнений:
$$
\begin{cases}
mx'' = -cx'  &(1) \\
my'' = -cy' - mg &(2) \\
x'(0) = v_0 \cos{\alpha} &(3) \\
y'(0) = v_0 \sin{\alpha} &(4) \\
x(0) = x_0 \\
y(0) = y_0 \\
y(x(t)) = \phi (x(t)) \\
\end{cases}
$$

## Решение

Решим систему уравнений аналитическим методом:

$$
z = x'
m \frac{dz}{dt} = -cz \\
\frac{dz}{z} = - \frac{c}{m} dt \\
\ln(z C_1) = - \frac{c}{m} t \\
z = e^{- \frac{c}{m} t} / C_1 \\
z(0) = v_0 \cos{\alpha} => \frac{1}{C_1} = v_0 \cos{\alpha} => C_1 = \frac{1}{\cos{\alpha}} => z = e^{- \frac{c}{m} t} v_0 \cos{\alpha}


$$

![](./Untitled1.png)

In [2]:
import numpy as np
import scipy.optimize as spo

# initial_coords = list(map(float, input('Введите начальные координаты ').split()))
x0, y0 = 0, 0
v0 = 10
theta = np.radians(45)
m = 1
c = 1
# v0 = float(input('Введите начальную скорость '))
# theta = np.radians(float(input('Введите угол броска над горизонтом ')))
# m = float(input('Введите массу тела '))
# c = float(input('Введите коэффициент сопротивления воздуха '))
g = 10
t_max = 3600000
phi = lambda x: 0
assert c != 0
res_func = lambda t: \
        y0 + m/c * ((m/c*g + v0*np.sin(theta)) * (1 - np.exp(-c/m*t)) - g*t) \
        - phi(x0 + m/c*v0*np.cos(theta)*(-np.exp(-c/m*t) + 1))
starting_estimate = 1
ans = spo.fsolve(res_func, 1)
while ans == 0:
    starting_estimate *= 2
    ans = spo.fsolve(res_func, 1)
    if starting_estimate > t_max / 2:
        print('Время полета не найдено ')
        break
print('Время полета:', ans)


Время полета: [1.18534593]


![](./Untitled2.png)

In [57]:
import numpy as np
import pandas as pd

In [63]:
# initial_coords = list(map(float, input('Введите начальные координаты ').split()))
x0, y0 = 0, 0
v0 = 10
theta = np.radians(45)
m = 1
c = 1
# v0 = float(input('Введите начальную скорость '))
# theta = np.radians(float(input('Введите угол броска над горизонтом ')))
# m = float(input('Введите массу тела '))
# c = float(input('Введите коэффициент сопротивления воздуха '))
g = 10
t_max = 3600000
phi = lambda x: np.sin(x)
assert c != 0

In [64]:
def f(t, y):
    dy0 = -c/m * y[0]
    dy1 = y[0]
    dy2 = -(c*y[2] + m*g) / m
    dy3 = y[2]
    return np.array([dy0, dy1, dy2, dy3])

In [65]:
t = 0.0
t_old = t
res = np.array([v0*np.cos(theta), x0, v0 * np.sin(theta), y0])
res_old = res
h = 0.0001
history = [(t, res[1], res[3])]
# np.append(history, (t, res[1], res[3]))
while t < t_max and res[3] >= phi(res[1]):
        t_old = t
        res_old = res.copy()
        k1 = f(t, res)
        k2 = f(t + h/2, res + h/2*k1)
        k3 = f(t + h/2, res + h/2*k2)
        k4 = f(t + h, res + h*k3)
        res += h/6 * (k1 + 2*k2 + 2*k3 + k4)
        t += h
        history.append((t, res[1], res[3]))
assert t < t_max
print(history[0])
print(history[-1])
print(history[-2])

(0.0, 0.0, 0.0)
(0.0001, 0.0007070714270259702, 0.000707021428692595)
(0.0, 0.0, 0.0)


In [66]:
# interpolation of `history`
def polinom(t, ts, xs):
    l = []
    for i in range(len(ts)):
        li = 1
        for j in range(0, i):
            li *= (t - ts[j]) / (ts[i] - ts[j])
        for j in range(i+1, len(ts)):
            li *= (t - ts[j]) / (ts[i] - ts[j])
        l.append(li)
    ll = 0.0
    for i in range(len(xs)):
        ll += l[i] * xs[i]
    return ll

In [67]:
ts, xs, ys = zip(*history[-5:])
t_before, x_before, y_before = history[-1]
t_after, x_after, y_after = history[-2]

eps = 0.00000001
def res_func(t):
    return polinom(t, ts, ys) - phi(polinom(t, ts, xs))

ans = t_after
ans_old = t_before
while np.abs(ans - ans_old) > eps:
    new_ans = ans_old - res_func(ans_old) * (ans - ans_old) / (res_func(ans) - res_func(ans_old))
    ans_old = ans
    ans = new_ans 
print('Время полета:', ans)
print('x:', polinom(ans, ts, xs))
print('y:', polinom(ans, ts, ys))


Время полета: 0.0
x: 0.0
y: 0.0
